using chroma

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("hugginglearners/netflix-shows", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
import re, nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.DataFrame(dataset)

In [5]:
def preprocess_text(text: str) -> str:
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text)

    # remove stopwords
    tokens = nltk.word_tokenize(text)
    text = " ".join(tokens)
    text = text.lower().strip()

    return text

df['description'] = df['description'].apply(lambda text: preprocess_text(text))
df = df[df['description'] != '']

In [6]:
df['description']

0       as her father nears the end of his life filmma...
1       after crossing paths at a party a cape town te...
2       to protect his family from a powerful drug lor...
3       feuds flirtations and toilet talk go down amon...
4       in a city of coaching centers known to train i...
                              ...                        
8802    a political cartoonist a crime reporter and a ...
8803    while living alone in a spooky town a young gi...
8804    looking to survive in a world taken over by zo...
8805    dragged from civilian life a former superhero ...
8806    a scrappy but poor boy worms his way into a ty...
Name: description, Length: 8807, dtype: object

In [7]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 50

In [ ]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key="YOUR_API_KEY",
    model_name="sentence-transformers/all-MiniLM-L6-v2" #choose model to use
)

In [8]:

def perform_search(query,count,file_path,exist_path):

    #load_csv here

    chroma_client = chromadb.PersistentClient(path=exist_path) #see if the chromaDB has already been created

    if count == 1:
        context = chroma_client.create_collection(name="context")
        context.add(
            ids=[str(i) for i in range(0, 100)],  # IDs are just strings
            documents=dataset["description"][:100],
            metadatas=[{"type": dataset["listed_in"][i]} for i in range(0, 100)
    ],
)
    else:
        context = chroma_client.get_collection(name="context") #if the vectorstore already exist

    results = context.query(
    query_texts=query,
    n_results=1
    )
    return results["documents"][0]


In [10]:
perform_search("give me a horror movie", 1,"","")

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:06<00:00, 13.0MiB/s]


["Equestria's divided. But a bright-eyed hero believes Earth Ponies, Pegasi and Unicorns should be pals — and, hoof to heart, she’s determined to prove it."]

In [14]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.4 MB/s eta 0:00:00


In [1]:
!pip install milvus langchain

In [12]:

from langchain.vectorstores import Milvus
from langchain_community.embeddings import HuggingFaceEmbeddings

In [3]:
#milvus
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [4]:
from milvus import default_server
default_server.start()
connections.connect("default", host = "127.0.0.1", port = default_server.listen_port)


In [15]:
vectordb = Milvus.from_documents(
    {},
    HuggingFaceEmbeddings(),
    connection_args = {"host": "127.0.0.1", "port": default_server.listen_port }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
from langchain.memory.vectorstore import VectorStoreRetrieverMemory
retriever = Milvus.as_retriever(vectordb, search_kwargs = dict(k=1))
memory = VectorStoreRetrieverMemory(retriever = retriever)

In [ ]:
#load_csv here
for data in dataframe:
  memory.save_context({},{})

In [ ]:
memory.load_memory_variables({})

In [ ]:
!pip install chromadb
!pip install langchain_community
import chromadb
from langchain_community.document_loaders import WebBaseLoader


def perform_search(query,count,file_path,exist_path):

    #load_csv here

    chroma_client = chromadb.PersistentClient(path=exist_path) #see if the chromaDB has already been created

    if count == 1:
        context = chroma_client.create_collection(name="context")
        for i in range(len(documents)):
            context.add(
            documents=documents[i],
            ids="id{}".format(i+1)
            #meta data here
            )
        for link in links:
            loader = WebBaseLoader(link)
            data = loader.load()
            context.add(
            documents=data[0].page_content,
            ids="id"
            )
    else:
        context = chroma_client.get_collection(name="context") #if the vectorstore already exist
    results = context.query(
    query_texts=query,
    n_results=1
    )
    return results["documents"][0]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
